# Imports

In [1]:
!pip install biopython

In [2]:
import numpy as np
import pandas as pd
from Bio import SeqIO
import tensorflow as tf
# tf.enable_eager_execution()
from tensorflow import keras
print('tf version:',tf.__version__)
print('keras version:',keras.__version__)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Settings

In [ ]:
test_fasta = 'data/test_13_classes.fasta'
train_fasta = 'data/train.fasta' 
val_fasta = 'data/val.fasta'

In [ ]:
cd 'drive/My Drive/Colab Notebooks'

# inspect data

In [ ]:
with open(test_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    classes = []

    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        seqs.append(seq_record.seq)
        classes.append(seq_record.description.split()[1])
        lengths.append(len(seq_record.seq))

    df = pd.DataFrame({
        'identifier':identifiers,
        'family':classes,
        'seq':seqs,
        'length':lengths         
        })

In [ ]:
df.head

In [ ]:
df.family.hist(figsize=(25,5))

# Load and Preprocess

In [ ]:
w_to_idx = {"<PAD>": 0, "A": 1, "G": 2, "C": 3, "T": 4, "N": 5, "M": 6, "R": 7, "Y": 8, "W": 9,
              "K": 10, "S": 11, "H": 12, "V": 13, "B": 14, "D": 15}
families = [
           '5S_rRNA',
           '5_8S_rRNA',
           'CD-box',
           'HACA-box',
           'IRES',
           'Intron_gpI',
           'Intron_gpII',
           'leader',
           'miRNA',
           'riboswitch',
           'ribozyme',
           'scaRNA',
           'tRNA'
           ]
fam_to_idx = {}
for i,f in enumerate(families):
    fam_to_idx[f] = i

In [ ]:
def read_file(fname, mapper=w_to_idx):

    with open(fname) as fasta_file:  # Will close handle cleanly
        identifiers = []
        lengths = []
        seqs = []
        classes = []

        for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
            identifiers.append(seq_record.id)
            seqs.append(seq_record.seq)
            classes.append(seq_record.description.split()[1])
            lengths.append(len(seq_record.seq))

    df = pd.DataFrame({
        'identifier':identifiers,
        'family':classes,
        'seq':seqs,
        'length':lengths         
        })

    x_idxss = [[w_to_idx[w] for w in seq][0:min(len(seq),750)]+[w_to_idx['<PAD>']]*(750 - len(seq)) for seq in seqs]
    y_idxs = [fam_to_idx[f] for f in classes]

    xs = tf.convert_to_tensor([tf.one_hot(idxs, len(w_to_idx)) for idxs in x_idxss])
    ys = tf.one_hot(y_idxs, len(families))
    
    return xs,ys,df
        
        

In [ ]:
xs_test, ys_test,df = read_file(test_fasta)
xs_train, ys_train,_ = read_file(train_fasta)
xs_val ,ys_val,df = read_file(val_fasta)

In [ ]:
df.family.hist(figsize=(25,5))

# Model

In [ ]:
from tensorflow.keras.layers import MaxPooling1D, Dropout, Dense, Flatten, Conv1D, BatchNormalization, Activation
from tensorflow.keras import Model, activations

In [ ]:
print('dimension of x',xs_train.shape)
print('dimension of y',ys_train.shape)

In [ ]:
class MyModel(Model):
    def __init__(self, nc=1, nd=1, nfilter=32, filter_size=10, dense_node=256):
        super(MyModel, self).__init__()
        self.nc = nc
        self.nd = nd
        # conv block
        self.c_conv = [Conv1D(nfilter, filter_size, activation='relu') for i in range(nc)]
        self.c_batchnorm = [BatchNormalization() for i in range(nc)]
        self.c_dropout =  [Dropout(0.2) for i in range(nc)]
        self.c_activation = [Activation(activations.relu) for i in range(nc)]
        self.c_maxpool = [MaxPooling1D(2,2) for i in range(nc)]
        
        self.flat = Flatten()
        
        # dense block
        self.d_dense = [Dense(dense_node, activation='relu') for i in range(nd)]
        self.d_batchnorm = [BatchNormalization() for i in range(nd)]
        self.d_dropout = [Dropout(0.2) for i in range(nd)]
        self.d_activation = [Activation(activations.relu) for i in range(nd)]

        # final result
        self.d1 = Dense(13)

    def call(self, x):
        for i in range(self.nc):
            x = self.c_conv[i](x)
            x = self.c_batchnorm[i](x)
            x = self.c_dropout[i](x)
            x = self.c_activation[i](x)
            x = self.c_maxpool[i](x)
            
        x = self.flat(x)
        
        for i in range(self.nd):
            x = self.d_dense[i](x)
            x = self.d_batchnorm[i](x)
            x = self.d_dropout[i](x)
            x = self.d_activation[i](x)

        return self.d1(x)

# model = MyModel(3,3)

In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(filters=8, kernel_size=(2,2)),
#     tf.keras.layers.Flatten(input_shape=(750, 16)),

#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(13)
# ])


## Categorical cross-entropy loss

In [ ]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# model.compile(optimizer='adam',
#               loss=loss_fn,
#               metrics=['accuracy'])
# history = model.fit(xs_train, ys_train, batch_size=128, epochs=15, callbacks=[callback])
# model.evaluate(xs_test,  ys_test, verbose=2)

In [ ]:
# model.evaluate(xs_test,  ys_test, verbose=2)

# Try tune it by our own parameter

In [ ]:
# len(epochs)*len(nds)*len(ncs)*len(nfilters)*len(filtersizes)*len(densenodes)

In [ ]:
epochs = range(5,6)
nds = range(1,2)
ncs = range(1,2)
nfilters = [8,9]
filtersizes = [2,4]
densenodes =[16,32]

In [ ]:
epochs = range(5,26)
nds = range(1,4)
ncs = range(1,4)
nfilters = [8,16,32,64,128,256,512]
filtersizes = [2,3,4,5,6,7,8,10,12,14,16,20]
densenodes =[16,32,64,128,256]

In [ ]:
def train(nepoch=6,nc=1,nd=1,nfilter=16,filter_size=4,dense_node=32):
    param = {'nepoch':nepoch,'nc':nc,'nd':nd,'nfilter':nfilter,'filter_size':filter_size,'dense_node':dense_node}
    print('param',param)
    model = MyModel(nc,nd,nfilter,filter_size,dense_node)
    model.compile(optimizer='adam',
                  loss=loss_fn,
                  metrics=['accuracy'])
    history = model.fit(xs_train, ys_train, batch_size=128, epochs=nepoch, 
                        validation_data=(xs_val, ys_val), callbacks=[callback])
    res = model.evaluate(xs_test,  ys_test, verbose=2)
    return history, res, param

In [ ]:

# histories = []
# evals = []
# params = []


# max_accu = 0
# max_epoch = 0
# for i in epochs:
#     history, res, param = train(nepoch = i)
#     histories.append(history)
#     evals.append(res)
#     params.append(param)
#     if res[1] > max_accu:
#         max_accu = res[1]
#         max_epoch = i

# max_accu = 0
# max_nd = 0
# for i in nds:
#     history, res, param = train(nepoch = max_epoch, nd=i)
#     histories.append(history)
#     evals.append(res)
#     params.append(param)
#     if res[1] > max_accu:
#         max_accu = res[1]
#         max_nd = i

# max_accu = 0
# max_nc = 0
# for i in ncs:
#     history, res, param = train(nepoch = max_epoch, nd=max_nd, nc=i)
#     histories.append(history)
#     evals.append(res)
#     params.append(param)
#     if res[1] > max_accu:
#         max_accu = res[1]
#         max_nc = i
        
# max_accu = 0
# max_nfilter = 0
# for i in nfilters:
#     history, res, param = train(nepoch = max_epoch, nd=max_nd, nc=max_nc, nfilter=i)
#     histories.append(history)
#     evals.append(res)
#     params.append(param)
#     if res[1] > max_accu:
#         max_accu = res[1]
#         max_nfilter = i

# max_accu = 0
# max_filtersizes = 0
# for i in filtersizes:
#     history, res, param = train(nepoch = max_epoch, nd=max_nd, nc=max_nc, nfilter=max_nfilter, filter_size=i)
#     histories.append(history)
#     evals.append(res)
#     params.append(param)
#     if res[1] > max_accu:
#         max_accu = res[1]
#         max_filtersizes = i
        
# max_accu = 0
# max_densenodes = 0
# for i in densenodes:
#     history, res, param = train(nepoch = max_epoch, nd=max_nd, nc=max_nc, nfilter=max_nfilter, filter_size=max_filtersizes, dense_node=i)
#     histories.append(history)
#     evals.append(res)
#     params.append(param)
#     if res[1] > max_accu:
#         max_accu = res[1]
#         max_densenodes = i
        


In [ ]:
# history, res, param = train(nepoch = 25, nd=2, nc=2, nfilter=8, filter_size=10)

In [ ]:
# history, res, param = train(nepoch = 25, nd=2, nc=2, nfilter=8, filter_size=16)

In [ ]:
history, res, param = train(nepoch = 1, nd=2, nc=2, nfilter=128, filter_size=20, dense_node=128)

## [fail] Exact implementation

In [ ]:


# ncrDeep = tf.keras.models.Sequential([
#     Conv1D(filters=512, kernel_size=20),
#     BatchNormalization(),
#     Dropout(0.2),
#     Activation(activations.relu),
#     MaxPooling1D(2,2),

#     Conv1D(filters=128, kernel_size=5),
#     BatchNormalization(),
#     Dropout(0.2),
#     Activation(activations.relu),
#     MaxPooling1D(2,2),

#     Flatten(),

#     Dense(128),
#     Dense(13),
#     Dropout(0.2),
#     Activation(activations.relu),

#     Dense(32),
#     Dropout(0.2),
#     Activation(activations.relu),
#     Dense(13)
# ])
# ncrDeep.compile(optimizer='adam',
#                   loss=loss_fn,
#                   metrics=['accuracy'])
# history = ncrDeep.fit(xs_train, ys_train, batch_size=128, epochs=19, 
#                         validation_data=(xs_val, ys_val), callbacks=[callback])
# res = modelncrDeep.evaluate(xs_test,  ys_test, verbose=2)

In [ ]:
# res = ncrDeep.evaluate(xs_test,  ys_test, verbose=2)

In [ ]:
# res